In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import time
import cv2 as cv

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.current_device())
##torch.cuda.set_device(1)
#print(torch.cuda.current_device())

Using cuda device
Device: cuda
Current cuda device: 0
Count of using GPUs: 2
0


In [2]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 20
batch_size = 4
epochs = 20000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        print(j)
        xs_0=list()
        for i in range(20): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            
            r=open ('data/'+str(i+20*j)+'_building_r_two_block.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g_two_block.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b_two_block.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float64)
            temp_greed=np.array(temp_green, dtype=np.float64)
            temp_blue=np.array(temp_blue, dtype=np.float64)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float64)
y_train = np.array(ys_0, dtype=np.float64)
#y_train = np.zeros_like(y_train)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()
#print(y_train)
xs_0=list()
xs_0_=list()
xs_1=list()

for j in range(data_number): 
    xs_0=list()
    for i in range(20): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+20*j)+'_r_two_block.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+20*j)+'_g_two_block.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+20*j)+'_b_two_block.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float64)
        temp_greed=np.array(temp_green, dtype=np.float64)
        temp_blue=np.array(temp_blue, dtype=np.float64)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float64)
x_test = torch.from_numpy(x_test).cuda()

print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_test = TensorDataset(x_test, y_train)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
#print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
torch.Size([20, 20, 3, 100, 200]) torch.Size([20, 19])
data_ready


In [ ]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,15,5,stride=2,dtype=torch.float64)
        self.layer_2 = nn.Conv2d(15,13,5,stride=2,dtype=torch.float64)
        self.layer_3 = nn.Conv2d(13,7,5,stride=2,dtype=torch.float64)
        self.layer_4 = nn.Conv2d(7,3,5,stride=2,dtype=torch.float64)
        self.layer_5 = nn.LSTM(81,40,2, batch_first = True,dtype=torch.float64)
        self.layer_6 = nn.Linear(40, 1,dtype=torch.float64)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        self.glob_flag=0
        
    def forward(self, x):
        x = x.reshape((-1, 3, 100, 200))
        x = self.layer_1(x)
        self.a = self.relu(x)
        x = self.layer_2(self.a)
        x = self.relu(x)
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.layer_4(x)
        x = self.relu(x)
        x = x.reshape((self.batch_size, self.length, x.shape[-3],x.shape[-2],x.shape[-1]))
        x = x.reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        x, (h_n, c_n) = self.layer_5(x)
        #print(x.shape,h_n.shape,c_n.shape)
        x = self.relu(x[:,1:,:])
        x = self.layer_6(x)
        x = torch.squeeze(x,dim=2)
        return x

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.999 ,
                                        last_epoch=-1,
                                        verbose=False)
def image_out(feature,batch,name):
    width_ = feature.grad.shape[-1]
    #print(torch.min(feature.grad.detach()))
    #result_temp = torch.sum(feature.grad.detach(),-1)
    #important
    result_temp = torch.sum(torch.abs(feature.grad.detach()),-1)
    #print(torch.where(result_temp>=0))
    height_ = result_temp.shape[-1]
    result_temp = torch.sum(result_temp,-1)
    channel=result_temp.shape[1]
    result_temp = result_temp/channel
    print(result_temp.shape)
    temp_ = feature.detach().reshape((model.batch_size*model.length, channel, height_,width_))
    #print(torch.min(temp_))
    #print(result_temp.shape,temp_.shape)
    for i in range(model.length):
        temp_sum=0
        for j in range(result_temp.shape[1]):
            #print(i,j,result_temp.shape[1])
            temp_sum += result_temp[batch*model.length+i,j]*(temp_[batch*model.length+i,j,:,:])
        #temp_sum=temp_sum
        temp_sum = temp_sum.detach().cpu().numpy()
        #print(np.min(temp_sum))
        temp_positive = np.where(temp_sum>0, temp_sum,0)
        temp_negative = np.where(temp_sum<0, temp_sum,0)
        
        #print(np.min(temp_sum))
        min_= np.min(temp_sum)
        temp_sum=temp_sum-min_
        max_ = np.max(temp_sum)
        result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'.png'
        pil_image.save(name_final)
        pil_image.close()

        max_ = np.max(temp_positive)
        result_ = np.around(((255*temp_positive)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'_positive.png'
        pil_image.save(name_final)
        pil_image.close()
        
        temp_negative=np.abs(temp_negative)
        max_ = np.max(temp_negative)
        result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'_negative.png'
        pil_image.save(name_final)
        pil_image.close()
    
def train(dataloader, model, loss, optimizer):
    loss_sum=0
    total_number=len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss_result = loss(pred, y)
        #print(pred.shape,y.shape)
        optimizer.zero_grad()
        if total_number == batch + 1:
            model.a.retain_grad()
        loss_result.backward()
        optimizer.step()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    
    print("train_loss",loss_sum)
    
    if model.glob_flag==10:
        image_out(model.a,0,"training_")
            
def test(dataloader, model, loss):
    loss_sum=0
    total_number=len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss_result = loss(pred, y)
        #print(pred.shape,y.shape)
        if total_number == batch + 1:
            optimizer.zero_grad()
            model.a.retain_grad()
        loss_result.backward()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    print("test_loss",loss_sum)
    if model.glob_flag==10:
        model.glob_flag=0
        image_out(model.a,0,"test_")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    test(dataloader_test, model, loss)
    model.glob_flag+=1
    scheduler.step()
    
print("Done!")

Epoch 1
-------------------------------
train_loss tensor(74.0259, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(73.2007, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 2
-------------------------------
train_loss tensor(72.4021, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(71.8389, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 3
-------------------------------
train_loss tensor(71.4199, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(70.8482, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 4
-------------------------------
train_loss tensor(70.5778, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(70.2931, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 5
-------------------------------
train_loss tensor(69.8537, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
tes

<ipython-input-4-109bb994dd12>:87: RuntimeWarning: divide by zero encountered in true_divide
  result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)
<ipython-input-4-109bb994dd12>:87: RuntimeWarning: invalid value encountered in true_divide
  result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)


tensor(65.3351, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
Epoch 12
-------------------------------
train_loss tensor(64.0399, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(64.0697, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 13
-------------------------------
train_loss tensor(63.2762, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(62.9730, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 14
-------------------------------
train_loss tensor(62.5475, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(62.3183, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 15
-------------------------------
train_loss tensor(61.7876, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(61.4758, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 16
--------------------

test_loss tensor(40.0279, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 49
-------------------------------
train_loss tensor(39.6333, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(39.5748, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 50
-------------------------------
train_loss tensor(39.3123, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(39.2465, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 51
-------------------------------
train_loss tensor(38.9671, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
test_loss tensor(38.9705, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
Epoch 52
-------------------------------
train_loss tensor(38.6474, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(38.7100, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>

test_loss tensor(28.8101, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 86
-------------------------------
train_loss tensor(28.7132, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(28.5949, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 87
-------------------------------
train_loss tensor(28.5054, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(28.3936, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 88
-------------------------------
train_loss tensor(28.3067, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(28.2020, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 89
-------------------------------
train_loss tensor(28.1133, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(28.0107, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 90
-------------------------------

...

train_loss tensor(11.8127, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.6741, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 308
-------------------------------
train_loss tensor(11.7909, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.6056, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 309
-------------------------------
train_loss tensor(11.7661, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.5161, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 310
-------------------------------
train_loss tensor(11.7208, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.5530, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 311
-------------------------------
train_loss tensor(11.6988, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
test_loss tens

train_loss tensor(10.7448, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.7242, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 345
-------------------------------
train_loss tensor(10.7313, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.6608, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 346
-------------------------------
train_loss tensor(10.6982, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.7530, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 347
-------------------------------
train_loss tensor(10.6893, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.6976, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 348
-------------------------------
train_loss tensor(10.6995, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.5709, device='c

train_loss tensor(9.8847, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
test_loss tensor(11.2623, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
Epoch 382
-------------------------------
train_loss tensor(9.8855, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.3770, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 383
-------------------------------
train_loss tensor(9.8319, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.1478, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 384
-------------------------------
train_loss tensor(9.8166, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.1039, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 385
-------------------------------
train_loss tensor(9.7936, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>

train_loss tensor(9.0626, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.8582, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 419
-------------------------------
train_loss tensor(9.0283, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.8104, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 420
-------------------------------
train_loss tensor(9.0095, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.7182, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 421
-------------------------------
train_loss tensor(8.9907, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
test_loss tensor(10.7302, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
torch.Size([80, 15])
Epoch 422
-------------------------------
train_loss tensor(8.9638, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>

train_loss tensor(8.3081, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.1958, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 456
-------------------------------
train_loss tensor(8.2909, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.1292, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 457
-------------------------------
train_loss tensor(8.2934, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.1176, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 458
-------------------------------
train_loss tensor(8.2574, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.1107, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 459
-------------------------------
train_loss tensor(8.2439, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(10.1500, device='cuda:0